## импорт библиотек

In [2]:
!pip install fake_useragent

In [4]:
import re
import time
import datetime
import pandas as pd
import warnings
from tqdm import tqdm
from bs4 import BeautifulSoup
# from selenium import webdriver
from dataclasses import dataclass
from fake_useragent import UserAgent
UserAgent().chrome
import requests
from google.colab import files

warnings.filterwarnings("ignore", category=FutureWarning)

##LENTA

In [13]:
res = requests.get('https://lenta.ru/2023/07/03/')
res.status_code

200

In [14]:
soup = BeautifulSoup(res.text)
# soup

In [15]:
news = soup.find_all('li', class_='archive-page__item')

print(len(news))
print(news)

31
[<li class="archive-page__item _article"><a class="card-full-other _archive" href="/articles/2023/07/03/crimean_war/"><div class="card-full-other__content"><div class="card-full-other__titles"><h3 class="card-full-other__title">«Мы всех всегда били, бьем и будем бить»</h3><span class="card-full-other__rightcol"> 170 лет назад началась Крымская война. Как она изменила Россию?</span></div><div class="card-full-other__info"><time class="card-full-other__info-item card-full-other__date">00:00, 3 июля 2023</time></div></div><div class="card-full-other__image-wrap"><img class="card-full-other__image" height="80" loading="lazy" src="https://icdn.lenta.ru/images/2023/05/23/13/20230523130904725/owl_article_120_1dd07be348fcb6db785e5f20fd82c453.jpg" width="120"/></div></a></li>, <li class="archive-page__item _article"><a class="card-full-other _archive" href="/articles/2023/07/03/metall/"><div class="card-full-other__content"><div class="card-full-other__titles"><h3 class="card-full-other__tit

In [69]:
# парсинг ссылок lenta
for el in news:
    if (el.find('h3', 'card-full-news__title')):
        title = el.find('h3', 'card-full-news__title').text
    else: title = 0
    print(title)

    if (el.find('a', 'card-full-news _archive')):
        link = el.find('a', 'card-full-news _archive').get('href')
    else:
        link = 0
    print(link)
    if (el.find('time', 'card-full-news__info-item card-full-news__date')):
        date = el.find('time', 'card-full-news__info-item card-full-news__date').text
    else: date = 0
    print(date)

    if (el.find('span', 'card-full-news__info-item card-full-news__rubric')):
        category = el.find('span', 'card-full-news__info-item card-full-news__rubric').text
    else:
        category = 0
    print(category)
    # print()

Уехавших в США российских шпажистов объявили в розыск
/news/2023/12/27/mvd/
06:32, 27 декабря 2023
Спорт
Иностранная волейболистка назвала поразившие ее в России вещи
/news/2023/12/27/inostrannaya-voleybolistka-nazvala-porazivshie-ee-v-rossii-veschi/
07:50, 27 декабря 2023
Спорт
Слуцкий возглавил футбольный клуб в Китае
/news/2023/12/27/new_cl/
08:58, 27 декабря 2023
Спорт
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Российских фигуристов пристыдили за участие в шоу накануне соревнований
/news/2023/12/27/rossiyskih-figuristov-pristydili-za-uchastie-v-shou-nakanune-sorevnovaniy/
09:29, 27 декабря 2023
Спорт
Вяльбе высказалась о поддержке российских атлетов иностранцами
/news/2023/12/27/vyalbe-vyskazalas-o-podderzhke-rossiyskih-atletov-inostrantsami/
10:12, 27 декабря 2023
Спорт
В ФФККР прокомментировали переход россиянки Самоделкиной в Казахстан
/news/2023/12/27/v-ffkkr-prokommentirovali-perehod-rossiyanki-samodelkinoy-v-kazahstan/
10:41, 27 декабря 2023
Спорт
Захарян вернулся в Росс

### РЕШЕНИЕ

In [16]:
# сохранение списка ссылок по заданным топикам (в примере парсятся все ссылки за декабрь 2023)

def get_all_links(pages):
    days = ['01', '02', '03', '04', '05', '06', '07', '05', '06', '07', '08', '09'] + list(range(10, 30))
    links_list = []

    for day in tqdm(days):

        for i in range(1, pages+1):
            URL = 'https://lenta.ru/2023/07/' + str(day) + '/page/' + str(i)

            res = requests.get(URL, headers={'User-Agent': UserAgent().chrome})
            soup = BeautifulSoup(res.text)
            time.sleep(0.33)
            # news = soup.find_all('li', class_='parts-page__item')
            news = soup.find_all('li', class_='archive-page__item')
            all_links = []
            topics = ['Россия', 'Экономика', 'Силовые структуры', 'Бывший СССР', 'Спорт', 'Забота о себе', 'Строительство', 'Путешествия', 'Наука и техника']

            # topics = '(Спорт)'
            pattern = r'https+'

            for topic in topics:
                for ref in news:
                    if (ref.find('a', 'card-full-news _archive')):
                        link = ref.find('a', 'card-full-news _archive').get('href')
                        if not re.findall(pattern, link):
                            if (ref.find('span', 'card-full-news__info-item card-full-news__rubric')) and (re.match(topic, ref.find('span', 'card-full-news__info-item card-full-news__rubric').text)):
                                all_links.append(link)
            all_links = set(all_links)
            all_full_links = list(map(lambda x: 'https://lenta.ru' + x, all_links))
            links_list += all_full_links
    return links_list

all_links = get_all_links(5) # здесь можно указать по какому кол-ву страниц будем перемещаться
print(len(all_links))
# print(all_links)
# сохраняем ссылки в файл
with open("lenta_links.txt", "w", encoding="utf-8") as f:
    for page in all_links:
        f.write("%s\n" % page)
files.download('lenta_links.txt') # скачивание файла

100%|██████████| 32/32 [20:57<00:00, 39.30s/it]

733


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
a = set(all_links)
len(a)

708

In [18]:
len(all_links )

733

In [20]:
# функция записи данных в датафрейм парсим нужные нам теги и сохраняем в датафрейм, если строк будет более 2000, то сохраняем файл
def get_lenta_news(all_links):
    lenta_news = pd.DataFrame()
    cnt = 0
    cnt_name = 0
    for link in tqdm(all_links):
        soup = BeautifulSoup(requests.get(link).text)
        time.sleep(0.3)
        if (soup.find('a', class_='topic-header__item topic-header__time')): date = soup.find('a', class_='topic-header__item topic-header__time').text
        if (soup.find('span', class_='topic-body__title')): title = soup.find('span', class_='topic-body__title').text.strip()
        if (soup.find('div', class_='topic-body__content')): text = soup.find('div', class_='topic-body__content').text.strip()
        if (soup.find('a', class_='topic-header__item topic-header__rubric')): category = soup.find('a', class_='topic-header__item topic-header__rubric').text
        if (soup.find('a', 'rubric-header__link _active')): sub_category = soup.find('a', 'rubric-header__link _active').text
        row = {'date': date, 'link': link, 'title': title, 'text': text, 'category' : category, 'sub_category': sub_category}
        lenta_news = pd.concat([lenta_news, pd.DataFrame([row])])
        cnt = cnt+1
        cnt_name = cnt_name+1
        if cnt > 1999:
            lenta_news.to_pickle(f"lenta{cnt_name}.df", compression="gzip")
            files.download(f'/content/lenta{cnt_name}.df')
            lenta_news = pd.DataFrame()
            cnt = 0
    return lenta_news.reset_index(drop=True)

In [21]:
lenta_df = get_lenta_news(all_links)
lenta_df.to_pickle("lenta.df", compression="gzip")
files.download('/content/lenta.df') # скачивание файла

100%|██████████| 733/733 [13:53<00:00,  1.14s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
lenta_df

,date,link,title,text,category,sub_category
0,"00:36, 1 июля 2023",https://lenta.ru/news/2023/07/01/pensions/,В России увеличились пенсии у трех категорий н...,Три категории российских пенсионеров с 1 июля ...,Экономика,Социальная сфера
1,"00:37, 1 июля 2023",https://lenta.ru/news/2023/07/01/rr/,В США провели учения с макетами С-300,В Западной Вирджинии провели учения с макетами...,Наука и техника,Оружие
2,"00:08, 1 июля 2023",https://lenta.ru/news/2023/06/30/nato_ukrain/,Кулеба назвал самоубийством отказ Европы приня...,Европа столкнется с угрозой вооруженного конфл...,Бывший СССР,Украина
3,"01:57, 1 июля 2023",https://lenta.ru/news/2023/07/01/trud/,В украинском городе «запретили труд»,В находящемся под контролем Киева Купянске «за...,Бывший СССР,Украина
4,"00:09, 1 июля 2023",https://lenta.ru/news/2023/06/30/muscles_/,Эксперт дала совет по набору мышечной массы дл...,Инженер-технолог пищевого производства Ирина Г...,Спорт,ЗОЖ и фитнес
...,...,...,...,...,...,...
728,"13:48, 29 июля 2023",https://lenta.ru/news/2023/07/29/izbegayem/,Украинским шахматистам рекомендовали избегать ...,Федерация шахмат Украины в своем Facebook (соц...,Спорт,Летние виды
729,"13:49, 29 июля 2023",https://lenta.ru/news/2023/07/29/rab/,Работа российских средств РЭБ в зоне СВО попал...,Работа российских средств радиоэлектронной бор...,Бывший СССР,Украина
730,"13:58, 29 июля 2023",https://lenta.ru/news/2023/07/29/otkaz/,Стало известно имя получившей отказ во въезде ...,Отказ во въезде в Чехию для участия в турнире ...,Спорт,Летние виды
731,"14:25, 29 июля 2023",https://lenta.ru/news/2023/07/29/vsu/,Российские войска нанесли удар по пункту управ...,Вечером 28 июля российские войска нанесли удар...,Бывший СССР,Украина


In [23]:
set(lenta_df.category)

{'Бывший СССР',
 'Забота о себе',
 'Наука и техника',
 'Силовые структуры',
 'Спорт',
 'Экономика'}

In [17]:
set(lenta_df.category)

{'Спорт'}

## Парсинг Fontanka.Ru

In [ ]:
def get_fontanka_links(month, day):
    url = f'https://www.fontanka.ru/stroy/2019/{month}/{day}/all.html'
    req = requests.get(url).text
    soup = BeautifulSoup(req)
    news = soup.find_all('li', class_='IXaf5')
    lenta_news = pd.DataFrame()

    if len(news) > 0:

        for el in news:
            if (el.find('a', 'IXdb')):
                title = el.find('a', 'IXdb').text
            else: title = 0
            if (el.find('a', 'IXdb')):
                link = el.find('a', 'IXdb').get('href')
            else:
                link = 0

            row = {'title': title, 'link': link}
            lenta_news = pd.concat([lenta_news, pd.DataFrame([row])])


        return lenta_news.reset_index(drop=True)
    else:
        row = {'title': 0, 'link': 0}
        lenta_news = pd.concat([lenta_news, pd.DataFrame([row])])
        return lenta_news.reset_index(drop=True)

def fontanka(text):
    if text[:4] == 'http':
        return text
    else:
        return 'https://www.fontanka.ru' + text

In [ ]:
days = ['01', '02', '03', '04', '05', '06', '07', '05', '06', '07', '08', '09'] + list(range(10, 30))
monthes = ['01', '02', '03', '04', '05', '06', '07', '05', '06', '07', '08', '09', 10, 11, 12]
news = pd.DataFrame()

for month in tqdm(monthes):
    for day in days:
        new = get_fontanka_links(month, day)
        news = pd.concat((news, new), axis=0)

    news.link = news.link.astype('str').apply(fontanka)
news = news.loc[((news['link'] != 'https://www.fontanka.ru0'))]

news

100%|██████████| 15/15 [08:21<00:00, 33.41s/it]


,title,link
0,Наш проект: Как обставить маленькую квартиру? ...,https://www.fontanka.ru/longreads/kak_obstavit...
1,За время пути квартиры могли подрасти.,https://www.fontanka.ru/2018/12/26/092/
0,Строительство новой ледовой арены СКА на месте...,https://www.fontanka.ru/2019/01/09/093/
1,"Наш проект с Группой «Эталон» : Кладовки, балк...",https://www.fontanka.ru/longreads/udobniye_pla...
2,В петербургский ЗакС внесли законопроект о пер...,https://www.fontanka.ru/2019/01/09/050/
...,...,...
0,Финский ЮИТ впервые с 2016 года приобрел землю...,https://www.fontanka.ru/2019/12/23/060/
0,На Васильевском острове воссоздали особняк Фел...,https://www.fontanka.ru/2019/12/25/057/
0,«Мы пережили это. Идем дальше».,https://www.fontanka.ru/longreads/mi_perezhili...
1,«Мы пережили это. Идем дальше».,https://www.fontanka.ru/longreads/mi_perezhili...


In [ ]:
news[news.duplicated()]

,title,link
0,"Плохо или еще хуже: В Смольном решают, как под...",https://www.fontanka.ru/2019/02/05/080/
1,Структура Glorax Development нарубила дров. Ущ...,https://www.fontanka.ru/2019/02/05/115/
0,"«Группа ЛСР», Setl Group, ""Эталон"" и «Главстро...",https://www.fontanka.ru/2019/02/06/044/
1,"Глава «Группы ЛСР», бывший министр транспорта ...",https://www.fontanka.ru/2019/02/06/029/
0,Гендиректором «СПб Реновации» назначен Андрей ...,https://www.fontanka.ru/2019/03/06/068/
...,...,...
0,«Фонтанка.ру» приглашает на круглый стол «Недв...,https://www.fontanka.ru/2019/12/02/069/
0,«Тут вообще райское место». Соседи EcoCity рас...,https://www.fontanka.ru/longreads/zhizn_v_malo...
1,«Фонтанка.ру» приглашает на круглый стол «Недв...,https://www.fontanka.ru/2019/12/02/069/
3,Наш проект: Море у порога. Зачем люди селятся ...,https://www.fontanka.ru/longreads/kvartira_na_...


In [ ]:
news = news.drop_duplicates(keep=False)
news

,title,link
0,Наш проект: Как обставить маленькую квартиру? ...,https://www.fontanka.ru/longreads/kak_obstavit...
1,За время пути квартиры могли подрасти.,https://www.fontanka.ru/2018/12/26/092/
0,Строительство новой ледовой арены СКА на месте...,https://www.fontanka.ru/2019/01/09/093/
1,"Наш проект с Группой «Эталон» : Кладовки, балк...",https://www.fontanka.ru/longreads/udobniye_pla...
2,В петербургский ЗакС внесли законопроект о пер...,https://www.fontanka.ru/2019/01/09/050/
...,...,...
0,Застройщики ускорили ввод жилья перед новым годом,https://www.fontanka.ru/2019/12/20/075/
0,Наш проект: Сможете ли вы обставить большую кв...,https://www.fontanka.ru/longreads/obstav_bolsh...
0,Финский ЮИТ впервые с 2016 года приобрел землю...,https://www.fontanka.ru/2019/12/23/060/
0,На Васильевском острове воссоздали особняк Фел...,https://www.fontanka.ru/2019/12/25/057/


In [ ]:
# links = news.link.values
# with open("fontanka_links.txt", "w", encoding="utf-8") as f:
#     for page in links:
#         f.write("%s\n" % page)
# files.download('fontanka_links.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
links = news.link.values
len(links)

230

In [ ]:
def get_fontanka_posts(all_links):
    fontanka_news = pd.DataFrame()
    for link in tqdm(all_links):
        if requests.get(link).status_code == 200:
            soup = BeautifulSoup(requests.get(link).text)
            time.sleep(0.3)
            if (soup.find_all('div', class_='CNah KTap KTah')):
                text = soup.find_all('div', class_='CNah KTap KTah')
                row = {'link': link, 'text': text}
                fontanka_news = pd.concat([fontanka_news, pd.DataFrame([row])])
        else:
            text = 0
            row = {'link': link, 'text': text}
            fontanka_news = pd.concat([fontanka_news, pd.DataFrame([row])])


    return fontanka_news.reset_index(drop=True)

In [ ]:
fontanka_df2019 = get_fontanka_posts(links)
fontanka_df2019
# # fontanka_df.to_pickle("fontanka_df2022.df", compression="gzip")
# # files.download('/content/fontanka_df.df')

100%|██████████| 230/230 [08:20<00:00,  2.18s/it]


,link,text
0,https://www.fontanka.ru/2018/12/26/092/,"[[[<b class=""_"">Новый для Петербурга застройщи..."
1,https://www.fontanka.ru/2019/01/09/093/,[[[\tЗаместитель председателя Совета директоро...
2,https://www.fontanka.ru/2019/01/09/050/,[[[\tДепутат городского парламента Александр Т...
3,https://www.fontanka.ru/2019/01/11/093/,[[[\tГБУ «Мостретрест» подвел итоги двух конку...
4,https://www.fontanka.ru/2019/01/18/103/,"[[[<b class=""_"">В одном из последних петербург..."
...,...,...
159,https://www.fontanka.ru/2019/12/19/154/,"[[[<b class=""_"">Активно открывать эскроу-счета..."
160,https://www.fontanka.ru/2019/12/19/130/,"[[[<b class=""_"">Сроки принятия решений по сдел..."
161,https://www.fontanka.ru/2019/12/20/075/,[[[\tПеред новым годом застройщики активно сда...
162,https://www.fontanka.ru/2019/12/23/060/,"[[[<b class=""_"">Финская компания «ЮИТ Санкт-Пе..."


In [ ]:
fontanka_df2019

,link,text
0,https://www.fontanka.ru/2018/12/26/092/,"[[[<b class=""_"">Новый для Петербурга застройщи..."
1,https://www.fontanka.ru/2019/01/09/093/,[[[\tЗаместитель председателя Совета директоро...
2,https://www.fontanka.ru/2019/01/09/050/,[[[\tДепутат городского парламента Александр Т...
3,https://www.fontanka.ru/2019/01/11/093/,[[[\tГБУ «Мостретрест» подвел итоги двух конку...
4,https://www.fontanka.ru/2019/01/18/103/,"[[[<b class=""_"">В одном из последних петербург..."
...,...,...
159,https://www.fontanka.ru/2019/12/19/154/,"[[[<b class=""_"">Активно открывать эскроу-счета..."
160,https://www.fontanka.ru/2019/12/19/130/,"[[[<b class=""_"">Сроки принятия решений по сдел..."
161,https://www.fontanka.ru/2019/12/20/075/,[[[\tПеред новым годом застройщики активно сда...
162,https://www.fontanka.ru/2019/12/23/060/,"[[[<b class=""_"">Финская компания «ЮИТ Санкт-Пе..."


### препроцессинг

In [ ]:
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
from nltk.tokenize import word_tokenize

import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
mystopwords = stopwords.words('russian') + ['фото', 'фонтанка', 'риа', 'ру',
    'это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д',
    'который','прошлый','сей', 'свой', 'наш', 'мочь', 'такой'
]

# ru_words = re.compile("[А-Яа-я]+" )
ru_words = re.compile("[а-яА-Я0-9]+")

# lemmatizer = nltk.WordNetLemmatizer()

def words_only(text):
    return " ".join(ru_words.findall(text))

# def lemmatize(text):
#     try:
#         # return  " ".join([m.parse(w)[0].normal_form for w in text.lower().split()])
#         return " ".join([lemmatizer.lemmatize(w) for w in text.lower().split()])
#     except:
#         return " "

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""


def preprocess(text):
    return remove_stopwords(words_only(text.lower()))

In [ ]:
fontanka_df2019['text_prep'] = fontanka_df2019.text.astype('str').progress_apply(words_only)

100%|██████████| 164/164 [00:00<00:00, 2108.83it/s]


In [ ]:
fontanka_df2019

,link,text,text_prep
0,https://www.fontanka.ru/2018/12/26/092/,"[[[<b class=""_"">Новый для Петербурга застройщи...",Новый для Петербурга застройщик сумел удивить ...
1,https://www.fontanka.ru/2019/01/09/093/,[[[\tЗаместитель председателя Совета директоро...,Заместитель председателя Совета директоров хок...
2,https://www.fontanka.ru/2019/01/09/050/,[[[\tДепутат городского парламента Александр Т...,Депутат городского парламента Александр Тетерд...
3,https://www.fontanka.ru/2019/01/11/093/,[[[\tГБУ «Мостретрест» подвел итоги двух конку...,ГБУ Мостретрест подвел итоги двух конкурсов на...
4,https://www.fontanka.ru/2019/01/18/103/,"[[[<b class=""_"">В одном из последних петербург...",В одном из последних петербургских жилищных ко...
...,...,...,...
159,https://www.fontanka.ru/2019/12/19/154/,"[[[<b class=""_"">Активно открывать эскроу-счета...",Активно открывать эскроу счета покупатели жиль...
160,https://www.fontanka.ru/2019/12/19/130/,"[[[<b class=""_"">Сроки принятия решений по сдел...",Сроки принятия решений по сделке на рынке ново...
161,https://www.fontanka.ru/2019/12/20/075/,[[[\tПеред новым годом застройщики активно сда...,Перед новым годом застройщики активно сдают жи...
162,https://www.fontanka.ru/2019/12/23/060/,"[[[<b class=""_"">Финская компания «ЮИТ Санкт-Пе...",Финская компания ЮИТ Санкт Петербург приобрела...


In [ ]:
df_fontanka = pd.DataFrame({'link' : fontanka_df2019.link, 'text' : fontanka_df2019.text_prep, 'target' : 6})
df_fontanka

,link,text,target
0,https://www.fontanka.ru/2018/12/26/092/,Новый для Петербурга застройщик сумел удивить ...,6
1,https://www.fontanka.ru/2019/01/09/093/,Заместитель председателя Совета директоров хок...,6
2,https://www.fontanka.ru/2019/01/09/050/,Депутат городского парламента Александр Тетерд...,6
3,https://www.fontanka.ru/2019/01/11/093/,ГБУ Мостретрест подвел итоги двух конкурсов на...,6
4,https://www.fontanka.ru/2019/01/18/103/,В одном из последних петербургских жилищных ко...,6
...,...,...,...
159,https://www.fontanka.ru/2019/12/19/154/,Активно открывать эскроу счета покупатели жиль...,6
160,https://www.fontanka.ru/2019/12/19/130/,Сроки принятия решений по сделке на рынке ново...,6
161,https://www.fontanka.ru/2019/12/20/075/,Перед новым годом застройщики активно сдают жи...,6
162,https://www.fontanka.ru/2019/12/23/060/,Финская компания ЮИТ Санкт Петербург приобрела...,6


In [ ]:
df_fontanka[df_fontanka.duplicated()]

,link,text,target
45,https://www.fontanka.ru/2019/03/13/072/,С 1 июля станет обязательным долевое строитель...,6
48,https://www.fontanka.ru/2019/03/13/072/,С 1 июля станет обязательным долевое строитель...,6
51,https://www.fontanka.ru/2019/03/13/072/,С 1 июля станет обязательным долевое строитель...,6
53,https://www.fontanka.ru/2019/03/22/107/,Схема обмена старой квартиры на новую становит...,6
107,https://www.fontanka.ru/2019/08/28/128/,Петербургский комитет по градостроительству и ...,6
109,https://www.fontanka.ru/2019/08/29/111/,Цены на стройматериалы с начала года выросли в...,6
115,https://www.fontanka.ru/2019/09/02/094/,0,6
122,https://www.fontanka.ru/2019/09/17/127/,В документации превентивно объявленного конкур...,6
130,https://www.fontanka.ru/2019/10/10/098/,Вячеслав Макаров пообещал свернуть стройку жил...,6
147,https://www.fontanka.ru/2019/11/22/073/,За 3 года панельное домостроение в Петербурге ...,6


In [ ]:
df_fontanka = df_fontanka.drop_duplicates(keep=False)
df_fontanka

,link,text,target
0,https://www.fontanka.ru/2018/12/26/092/,Новый для Петербурга застройщик сумел удивить ...,6
1,https://www.fontanka.ru/2019/01/09/093/,Заместитель председателя Совета директоров хок...,6
2,https://www.fontanka.ru/2019/01/09/050/,Депутат городского парламента Александр Тетерд...,6
3,https://www.fontanka.ru/2019/01/11/093/,ГБУ Мостретрест подвел итоги двух конкурсов на...,6
4,https://www.fontanka.ru/2019/01/18/103/,В одном из последних петербургских жилищных ко...,6
...,...,...,...
159,https://www.fontanka.ru/2019/12/19/154/,Активно открывать эскроу счета покупатели жиль...,6
160,https://www.fontanka.ru/2019/12/19/130/,Сроки принятия решений по сделке на рынке ново...,6
161,https://www.fontanka.ru/2019/12/20/075/,Перед новым годом застройщики активно сдают жи...,6
162,https://www.fontanka.ru/2019/12/23/060/,Финская компания ЮИТ Санкт Петербург приобрела...,6


In [ ]:
df_fontanka.to_pickle("fontanka_df2019.df", compression="gzip")


## РИА НОВОСТИ СПОРТ


In [ ]:
res = requests.get('https://rsport.ria.ru/football')

res.status_code

200

In [ ]:
soup = BeautifulSoup(res.text)
soup



In [ ]:
news = soup.find_all('div', class_='list-item')

print(len(news))
print(news)

12
[<div class="list-item"><span class="schema_org" itemprop="itemListElement" itemscope="" itemtype="http://schema.org/ListItem"><meta content="1" itemprop="position"/><a href="https://rsport.ria.ru/20240224/zakharyan-1929291811.html" itemprop="url"> </a><meta content="Представитель Захаряна рассказал о травме игрока" itemprop="name"/></span><div class="list-item__content"><a class="list-item__image" href="https://rsport.ria.ru/20240224/zakharyan-1929291811.html"><picture><source media="(min-width: 1160px)" media-type="ar16x9" srcset="https://cdnn21.img.ria.ru/images/07e7/0a/15/1904373916_0:74:1080:682_203x0_80_0_0_83ff7c1b60ab0b2d45191ce9f38c046f.jpg"/><source media="(min-width: 480px)" media-type="ar16x9" srcset="https://cdnn21.img.ria.ru/images/07e7/0a/15/1904373916_0:74:1080:682_260x0_80_0_0_4bf8eeac2fce2d38fef544c495a8b6f4.jpg"/><source media="(min-width: 0px)" media-type="ar4x3" srcset="https://cdnn21.img.ria.ru/images/07e7/0a/15/1904373916_0:53:1080:863_180x0_80_0_0_95c7eb435e2

In [ ]:
news[1]

<div class="list-item__content"><a class="list-item__image" href="https://rsport.ria.ru/20240224/zenit-1929288153.html"><picture><source media="(min-width: 1160px)" media-type="ar16x9" srcset="https://cdnn21.img.ria.ru/images/07e8/01/0d/1921168123_0:0:3074:1729_203x0_80_0_0_10116e62f8b39a5d5cf7a891ee9f2c55.jpg"/><source media="(min-width: 480px)" media-type="ar16x9" srcset="https://cdnn21.img.ria.ru/images/07e8/01/0d/1921168123_0:0:3074:1729_260x0_80_0_0_8b3d5a6a0372c820e0be92922b728c66.jpg"/><source media="(min-width: 0px)" media-type="ar4x3" srcset="https://cdnn21.img.ria.ru/images/07e8/01/0d/1921168123_343:0:3074:2048_180x0_80_0_0_79a9766c189d67a5ac513d545a1c3d95.jpg"/><img alt="Футбол. РПЛ. Матч Динамо - Зенит" class="responsive_img m-list-img" data-crop-height="338" data-crop-ratio="0.5625" data-crop-width="600" data-responsive0="https://cdnn21.img.ria.ru/images/07e8/01/0d/1921168123_343:0:3074:2048_180x0_80_0_0_79a9766c189d67a5ac513d545a1c3d95.jpg" data-responsive1160="https://cd

In [ ]:
# lenta
for el in news:
    # if (el.find('')):
    #     title = el.find('a', 'list-item__title color-font-hover-only').title
    # else: title = 0
    # print(title)

    if (el.find('a')):
        link = el.find('a', 'list-item__title color-font-hover-only').get('href')
    else:
        link = 0
    print(link)

    # if (el.find('time', 'card-full-news__info-item card-full-news__date')):
    #     date = el.find('time', 'card-full-news__info-item card-full-news__date').text
    # else: date = 0
    # print(date)

    if (el.find('span', 'list-tag__text')):
        category = el.find('span', 'list-tag__text').text
    else:
        category = 0

    print(category)
    # print()

https://rsport.ria.ru/20240224/zakharyan-1929291811.html
Футбол
https://rsport.ria.ru/20240224/zenit-1929288153.html
Футбол
https://rsport.ria.ru/20240224/liverpul-1929277660.html
Футбол
https://rsport.ria.ru/20240224/futbol-1929269849.html
Футбол
https://rsport.ria.ru/20240224/futbol-1929269031.html
Футбол
https://rsport.ria.ru/20240224/futbol-1929267307.html
Футбол
https://rsport.ria.ru/20240224/sochi-1929265494.html
Футбол
https://rsport.ria.ru/20240224/lokomotiv-1929257082.html
Футбол
https://rsport.ria.ru/20240224/futbol-1929256245.html
Футбол
https://rsport.ria.ru/20240224/modrich-1929255377.html
Футбол
https://rsport.ria.ru/20240224/futbol-1929253226.html
Футбол
https://rsport.ria.ru/20240224/messi-1929241945.html
Футбол


In [ ]:
res = requests.get('https://rsport.ria.ru/football', headers={'User-Agent': UserAgent().chrome})
soup = BeautifulSoup(res.text)
news = soup.find_all('div', class_='list-item')
for ref in news:
    if (ref.find('a')):
        link = el.find('a', 'list-item__title color-font-hover-only').get('href')
    else:
        link = 0
        print(link)

In [ ]:
# сохранение списка ссылок по заданным топикам

def get_all_links():
    topics = ['football', 'figure_skating', 'biathlon', 'hockey', 'auto', 'fights', 'tennis']
    all_links = []


    for topic in tqdm(topics):
        URL = 'https://rsport.ria.ru/' + topic
        print(URL)
        res = requests.get(URL, headers={'User-Agent': UserAgent().chrome})
        soup = BeautifulSoup(res.text)
        time.sleep(0.33)
        news = soup.find_all('div', class_='list-item')


        for ref in news:
            if (ref.find('a')):
                link = ref.find('a', 'list-item__title color-font-hover-only').get('href')
            else:
                link = 0
            all_links.append(link)


    return all_links

all_links = get_all_links()


  0%|          | 0/7 [00:00<?, ?it/s]

https://rsport.ria.ru/football


 14%|█▍        | 1/7 [00:02<00:12,  2.15s/it]

https://rsport.ria.ru/figure_skating


 29%|██▊       | 2/7 [00:03<00:09,  1.87s/it]

...

 71%|███████▏  | 5/7 [00:08<00:03,  1.60s/it]

https://rsport.ria.ru/fights


 86%|████████▌ | 6/7 [00:09<00:01,  1.61s/it]

https://rsport.ria.ru/tennis


100%|██████████| 7/7 [00:11<00:00,  1.64s/it]


In [ ]:
len(all_links)

92

In [ ]:
a = set(all_links)
len(a)

92

In [ ]:
# функция записи данных в датафрейм
def get_lenta_news(all_links):
    lenta_news = pd.DataFrame()
    cnt = 0
    cnt_name = 0
    for link in tqdm(all_links):
        soup = BeautifulSoup(requests.get(link).text)
        time.sleep(0.3)
        if (soup.find('a', class_='topic-header__item topic-header__time')): date = soup.find('a', class_='topic-header__item topic-header__time').text
        if (soup.find('span', class_='topic-body__title')): title = soup.find('span', class_='topic-body__title').text.strip()
        if (soup.find('div', class_='topic-body__content')): text = soup.find('div', class_='topic-body__content').text.strip()
        if (soup.find('a', class_='topic-header__item topic-header__rubric')): category = soup.find('a', class_='topic-header__item topic-header__rubric').text
        if (soup.find('a', 'rubric-header__link _active')): sub_category = soup.find('a', 'rubric-header__link _active').text
        row = {'date': date, 'link': link, 'title': title, 'text': text, 'category' : category, 'sub_category': sub_category}
        lenta_news = pd.concat([lenta_news, pd.DataFrame([row])])
        cnt = cnt+1
        cnt_name = cnt_name+1
        if cnt > 1999:
            lenta_news.to_pickle(f"lenta{cnt_name}.df", compression="gzip")
            files.download(f'/content/lenta{cnt_name}.df')
            lenta_news = pd.DataFrame()
            cnt = 0
    return lenta_news.reset_index(drop=True)

In [ ]:
lenta_df = get_lenta_news(all_links)
lenta_df.to_pickle("lenta.df", compression="gzip")
files.download('/content/lenta.df')

100%|██████████| 701/701 [17:41<00:00,  1.51s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Быстрый парсер lenta через API

По мотивам ноутбука https://www.kaggle.com/code/hardtype/parsing-news-from-rbc-lenta-ru

In [1]:
# Установка библиотек
!pip install bs4
!pip install openpyxl

In [41]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

In [42]:
class lentaRu_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])

        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])

        return search_table


    def get_articles(self,
                     param_dict,
                     time_step = 37,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from '\
                  + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = self._get_search_table(param_copy)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel("lenta_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

In [43]:
# Задаем тут параметры
query = ''
offset = 0
size = 500
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "0" # topic = тематика новости
dateFrom = '2023-01-01'
dateTo = "2024-06-30"

param_dict = {'query'     : query,
              'from'      : str(offset),
              'size'      : str(size),
              'dateFrom'  : dateFrom,
              'dateTo'    : dateTo,
              'sort'      : sort,
              'title_only': title_only,
              'type'      : material,
              'bloc'      : bloc,
              'domain'    : domain}

print("param_dict:", param_dict)

param_dict: {'query': '', 'from': '0', 'size': '500', 'dateFrom': '2023-01-01', 'dateTo': '2024-06-30', 'sort': '3', 'title_only': '0', 'type': '0', 'bloc': '0', 'domain': '1'}


In [44]:
param_dict

{'query': '',
 'from': '0',
 'size': '500',
 'dateFrom': '2023-01-01',
 'dateTo': '2024-06-30',
 'sort': '3',
 'title_only': '0',
 'type': '0',
 'bloc': '0',
 'domain': '1'}

In [45]:
# Тоже будем собирать итеративно, правда можно ставить time_step побольше, т.к.
# больше лимит на запрос статей. И Работает быстрее :)

parser = lentaRu_parser()

tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 57,
                         save_every = 5,
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Checkpoint saved!
Finish
500


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,1635703,https://lenta.ru/news/2024/06/06/v-meksike-zaf...,В Мексике зафиксировали первый в мире случай з...,1717632653,1717632653,1,1,0,0,5,[195],https://icdn.lenta.ru/images/2024/06/06/03/202...,1717632653,Фото: Tatyana Makeyeva / Reuters Сергей Грехов...,В Мексике зафиксировали первый в мире случай з...,Фото: Tatyana Makeyeva / Reuters Сергей ... ор...
1,1635707,https://lenta.ru/news/2024/06/06/dron-vsu-atak...,Дрон ВСУ атаковал еще одну нефтебазу на террит...,1717632804,1717632836,1,1,0,1,1,[4],https://icdn.lenta.ru/images/2024/06/06/03/202...,1717632804,Фото: Globallookpress.com Беспилотник Вооружен...,Дрон ВСУ атаковал еще одну нефтебазу на террит...,Фото: Globallookpress.com Беспилотник ... одну...
2,1635708,https://lenta.ru/news/2024/06/06/v-rostovskoy-...,В Ростовской области сообщили об отсутствии по...,1717633175,1717633236,1,1,0,1,1,[4],https://icdn.lenta.ru/images/2024/06/06/03/202...,1717633175,Фото: MOD Russia / Globallookpress.com Пожар н...,В Ростовской области сообщили об отсутствии по...,Фото: MOD Russia / Globallookpress.com ... мет...
3,1635709,https://lenta.ru/news/2024/06/06/v-hersonskoy-...,В Херсонской области заявили о доказанной вине...,1717633350,1717633351,1,1,0,1,3,[55],https://icdn.lenta.ru/images/2024/06/06/03/202...,1717633350,Фото: Cover Images / Globallookpress.com Марин...,В Херсонской области заявили о доказанной вине...,Фото: Cover Images / Globallookpress.com ... в...
4,1634616,https://lenta.ru/news/2024/06/06/razdavavshaya...,Раздававшая микрокредиты контора выложила в се...,1717633831,1717633831,1,1,0,0,9,"[4, 42]",https://icdn.lenta.ru/images/2024/06/04/15/202...,1717633831,Фото: oneinchpunch / Shutterstock / Fotodom ...,Раздававшая микрокредиты контора выложила в се...,Фото: oneinchpunch / Shutterstock / ... и выкл...


In [46]:
tbl.to_csv("Lenta_sample.csv", index=False)

In [ ]:
# tbl = pd.read_csv("Lenta_sample.csv")

In [47]:
tbl.shape

(500, 16)

In [48]:
tbl['bloc'].value_counts(normalize=True)

bloc
4     0.228
1     0.146
37    0.130
3     0.122
2     0.118
8     0.042
5     0.034
7     0.032
47    0.030
0     0.026
9     0.024
48    0.022
87    0.020
6     0.016
12    0.006
86    0.002
49    0.002
Name: proportion, dtype: float64

Найдем соответствие между кодом блока, его названием и кодом в соревновании:

* 1 - Россия - 0
* 37 - Силовые структуры - 2
* 3 - Бывший СССР - 3
* 4 - Экономика - 1
* 5 - Наука и техника - 8
* 8 - Спорт - 4
* 48 - Туризм - 7
* 87 - Здоровье - 5

In [49]:
tbl[tbl.bloc == 3].iloc[0]

docid                                                    1635709
url            https://lenta.ru/news/2024/06/06/v-hersonskoy-...
title          В Херсонской области заявили о доказанной вине...
modified                                              1717633350
lastmodtime                                           1717633351
type                                                           1
domain                                                         1
status                                                         0
part                                                           1
bloc                                                           3
tags                                                        [55]
image_url      https://icdn.lenta.ru/images/2024/06/06/03/202...
pubdate                                               1717633350
text           Фото: Cover Images / Globallookpress.com Марин...
rightcol       В Херсонской области заявили о доказанной вине...
snippet        Фото: Cove

In [50]:
tbl = tbl[tbl.bloc.isin([1, 37, 3, 4, 5, 8, 48, 87])]

TagsMap = {1 : 0, 3 : 3, 4 : 1, 5 : 8, 8 : 4, 37 : 2, 48 : 7, 87 : 5}

tbl['topic'] = tbl['bloc'].map(TagsMap)

<ipython-input-50-5f5bf2ec18e9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl['topic'] = tbl['bloc'].map(TagsMap)


In [51]:
tbl.shape

(372, 17)

In [52]:
tbl['topic'].value_counts(normalize=True) # можно сверить с распределением меток классов в соревновании

topic
1    0.306452
0    0.196237
2    0.174731
3    0.163978
4    0.056452
8    0.045699
7    0.029570
5    0.026882
Name: proportion, dtype: float64